In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re, os
import numpy as np
import xarray as xr
import tensorflow.keras as keras
import datetime
import pdb
import matplotlib.pyplot as plt
from src.utils import *
from src.score import *
from src.data_generator import *
from src.networks import *
from src.train import *

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]=str(0)
limit_mem()

In [12]:
valdir = '/data/stephan/WeatherBench/5.625deg/'

In [13]:
z500_valid = load_test_data(f'{valdir}geopotential_500', 'z')
t850_valid = load_test_data(f'{valdir}temperature_850', 't')

In [14]:
!ls ../nn_configs/B/*resnet_d3_cmip*

../nn_configs/B/66-resnet_d3_cmip_MPI-ESM.yml
../nn_configs/B/67-resnet_d3_cmip_MPI-ESM_ext.yml
../nn_configs/B/68-resnet_d3_cmip_MIROC6.yml
../nn_configs/B/69-resnet_d3_cmip_MIROC6_ext.yml
../nn_configs/B/72-resnet_d3_cmip_M2.yml
../nn_configs/B/73-resnet_d3_cmip_M2_ext.yml


In [15]:
args = {}
args = load_args('../nn_configs/B/63-resnet_d3_best.yml')

In [16]:
mean = xr.open_dataarray(f"{args['model_save_dir']}/{args['exp_id']}_mean.nc")
std = xr.open_dataarray(f"{args['model_save_dir']}/{args['exp_id']}_std.nc")

In [20]:
args['ext_mean'] = mean
args['ext_std'] = std
args['test_years'] = ['2017', '2017']

In [21]:
dg_test = load_data(**args, only_test=True)

DG start 21:39:15.395913
DG normalize 21:39:15.422620
DG load 21:39:15.441338
Loading data into RAM
DG done 21:39:23.102115


In [23]:
def get_scores(exp_id):
    model = keras.models.load_model(
        f'/home/rasp/data/myWeatherBench/predictions/saved_models/{exp_id}.h5',
        custom_objects={'PeriodicConv2D': PeriodicConv2D, 'lat_mse': tf.keras.losses.mse}
    )
    preds = create_predictions(model, dg_test)
    print(compute_weighted_rmse(
        preds.z.sel(level=500) if hasattr(preds, 'level') else preds.z, z500_valid
    ).load())
    print(compute_weighted_rmse(
        preds.t.sel(level=850) if hasattr(preds, 'level') else preds.t, t850_valid
    ).load())

In [24]:
get_scores('66-resnet_d3_cmip_MPI-ESM')

<xarray.DataArray 'z_rmse' ()>
array(382.86692906)
Coordinates:
    level    int32 500
<xarray.DataArray 't_rmse' ()>
array(2.24228372)
Coordinates:
    level    int32 850


In [25]:
get_scores('67-resnet_d3_cmip_MPI-ESM_ext')

<xarray.DataArray 'z_rmse' ()>
array(337.74384929)
Coordinates:
    level    int32 500
<xarray.DataArray 't_rmse' ()>
array(2.10604679)
Coordinates:
    level    int32 850


In [26]:
get_scores('68-resnet_d3_cmip_MIROC6')

<xarray.DataArray 'z_rmse' ()>
array(461.67684631)
Coordinates:
    level    int32 500
<xarray.DataArray 't_rmse' ()>
array(2.8312586)
Coordinates:
    level    int32 850


In [27]:
get_scores('69-resnet_d3_cmip_MIROC6_ext')

<xarray.DataArray 'z_rmse' ()>
array(359.38601098)
Coordinates:
    level    int32 500
<xarray.DataArray 't_rmse' ()>
array(2.63872553)
Coordinates:
    level    int32 850


In [28]:
get_scores('72-resnet_d3_cmip_M2')

<xarray.DataArray 'z_rmse' ()>
array(369.32633307)
Coordinates:
    level    int32 500
<xarray.DataArray 't_rmse' ()>
array(2.22884998)
Coordinates:
    level    int32 850


In [29]:
get_scores('73-resnet_d3_cmip_M2_ext')

<xarray.DataArray 'z_rmse' ()>
array(327.74158608)
Coordinates:
    level    int32 500
<xarray.DataArray 't_rmse' ()>
array(2.07897274)
Coordinates:
    level    int32 850


In [31]:
model = keras.models.load_model(
        f'/home/rasp/data/myWeatherBench/predictions/saved_models/73-resnet_d3_cmip_M2_ext.h5',
        custom_objects={'PeriodicConv2D': PeriodicConv2D, 'lat_mse': tf.keras.losses.mse}
    )

In [39]:
len(model.layers)

177

In [36]:
[l.name for l in model.layers].index('add_14')

139

In [40]:
model.compile('adam', 'mae')

In [42]:
model.loss_functions